In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords, inaugural
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

In [2]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [3]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [4]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [5]:
def build_sentiment_model(): #might not need as function, likely calling this code only once
    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]
    
    dataset = positive_dataset + negative_dataset
    
    random.shuffle(dataset)
    
    train_data = dataset[:7000]
    test_data = dataset[7000:]
    classifier = NaiveBayesClassifier.train(train_data)
    
    return classifier

In [6]:
def positivity_score(filename, classifier):
    positive_sentences = 0
    negative_sentences = 0
    for sent in sent_tokenize(filename):
        custom_tokens = remove_noise(word_tokenize(sent))
        if(classifier.classify(dict([token, True] for token in custom_tokens)) == 'Positive'):
            positive_sentences += 1
        else:
            negative_sentences += 1
    return positive_sentences/(positive_sentences+negative_sentences)  

In [16]:
classifier = build_sentiment_model()
for i in inaugural.fileids():
    print(i, positivity_score(inaugural.raw(i),classifier))

1789-Washington.txt 0.8260869565217391
1793-Washington.txt 1.0
1797-Adams.txt 0.8918918918918919
1801-Jefferson.txt 0.6585365853658537
1805-Jefferson.txt 0.7555555555555555
1809-Madison.txt 0.8571428571428571
1813-Madison.txt 0.5757575757575758
1817-Monroe.txt 0.6229508196721312
1821-Monroe.txt 0.689922480620155
1825-Adams.txt 0.6216216216216216
1829-Jackson.txt 0.8
1833-Jackson.txt 0.6896551724137931
1837-VanBuren.txt 0.7789473684210526
1841-Harrison.txt 0.638095238095238
1845-Polk.txt 0.6274509803921569
1849-Taylor.txt 0.7727272727272727
1853-Pierce.txt 0.6730769230769231
1857-Buchanan.txt 0.7303370786516854
1861-Lincoln.txt 0.6370370370370371
1865-Lincoln.txt 0.6153846153846154
1869-Grant.txt 0.85
1873-Grant.txt 0.627906976744186
1877-Hayes.txt 0.7796610169491526
1881-Garfield.txt 0.6576576576576577
1885-Cleveland.txt 0.7272727272727273
1889-Harrison.txt 0.7643312101910829
1893-Cleveland.txt 0.7586206896551724
1897-McKinley.txt 0.7307692307692307
1901-McKinley.txt 0.58
1905-Roosevel